In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config spec.image = "mlrun/mlrun"

%nuclio: setting spec.image to 'mlrun/mlrun'


In [11]:
import os
import pandas as pd
import requests
import json
from datetime import datetime

def model_server_tester(context,
                      table: str,
                      label_column: str = "label",
                      addr='', 
                      model='',
                      match_err=False,
                      rows=10):
    """ Test a model server """
    
    
    table = str(table)
    if table.endswith(".csv"):
        table = pd.read_csv(table)
    else: 
        table = pd.read_parquet(table)

    y_list = table.pop(label_column).values.tolist()
    context.logger.info(f'testing with dataset against {addr}, model: {model}')
    if rows:
        table = table.sample(rows)
    
    count = err_count = match = total_time = 0
    for x, y in zip(table.values, y_list):
        count += 1
        event_data = json.dumps({"instances":[x.tolist()]})
        had_err = False
        try:
            start = datetime.now()
            resp = requests.put(f'{addr}/{model}/predict', json=event_data)
            if not resp.ok:
                context.logger.error(f'bad function resp!!\n{resp.text}')
                err_count += 1
                continue
            total_time += (datetime.now()-start).microseconds
                
        except OSError as err:
            context.logger.error(f'error in request, data:{event_data}, error: {err}')
            err_count += 1
            continue
            
        y_resp = resp.json()[0]
        if y == y_resp:
            match += 1
        
    context.log_result('total_tests', count)
    context.log_result('errors', err_count)
    context.log_result('match', match)
    if count - err_count > 0:
        context.log_result('avg_latency', int(total_time / (count - err_count)))
    context.logger.info(f'run {count} tests, {err_count} errors and {match} match expected value')
    
    if err_count:
        raise ValueError(f'failed on {err_count} tests of {count}')
    
    if match_err and match != count:
        raise ValueError(f'only {match} results match out of {count}')

In [4]:
# nuclio: end-code
# marks the end of a code section

In [5]:
from os import path
from mlrun import run_local, NewTask, mlconf, import_function, mount_v3io
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'

# specify artifacts target location
artifact_path = mlconf.artifact_path or path.abspath('./')
project_name = 'sk-project'

In [12]:
# run the function locally
addr = 'http://13.58.191.176:30115'
gen = run_local(name='model_server_tester', handler=model_server_tester, 
                params={'addr': addr, 'model': 'iris_dataset_v1'},
                inputs={'table': 'iris_dataset.parquet'},
                project=project_name, artifact_path=path.join(artifact_path, 'data')) 

[mlrun] 2020-04-06 23:26:00,746 starting run model_server_tester uid=b293663098374087b0bdd4d1e24bca86  -> http://10.196.88.27:80
[mlrun] 2020-04-06 23:26:00,915 testing with dataset against http://13.58.191.176:30115, model: iris_dataset_v1
[mlrun] 2020-04-06 23:26:01,137 run 10 tests, 0 errors and 1 match expected value



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
sk-project,...e24bca86,0,Apr 06 23:26:00,completed,model_server_tester,v3io_user=adminkind=handlerowner=adminhost=jupyter-74f9488695-mqhsp,table,addr=http://13.58.191.176:30115model=iris_dataset_v1,total_tests=10errors=0match=1avg_latency=21697,


to track results use .show() or .logs() or in CLI: 
!mlrun get run b293663098374087b0bdd4d1e24bca86 --project sk-project , !mlrun logs b293663098374087b0bdd4d1e24bca86 --project sk-project
[mlrun] 2020-04-06 23:26:01,426 run executed, status=completed


In [13]:
from mlrun import code_to_function
test_func = code_to_function(name='model_server_tester', kind='job')
test_func.spec.default_handler = "model_server_tester"
test_func.spec.description = "test model servers"
test_func.metadata.categories = ["models", "testing"]
test_func.metadata.labels = {"author": "yaronh"}

test_func.export('function.yaml')

[mlrun] 2020-04-06 23:27:38,885 function spec saved to path: function.yaml
